# Assignment 3.2

In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## 1. The dataset

In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3449,6.6,0.15,0.32,6.0,0.033,59.0,128.0,0.99192,3.19,0.71,12.1,8
4600,8.3,0.22,0.34,1.1,0.043,20.0,116.0,0.99270,3.00,0.47,10.2,6
1722,7.5,0.40,1.00,19.5,0.041,33.0,148.0,0.99770,3.24,0.38,12.0,6
1893,6.8,0.24,0.31,18.3,0.046,40.0,142.0,1.00000,3.30,0.41,8.7,5
2427,8.1,0.34,0.28,7.5,0.040,70.0,230.0,0.99558,3.14,0.55,9.8,6
362,6.8,0.15,0.30,5.3,0.050,40.0,127.0,0.99420,3.40,0.39,9.7,6
140,6.3,0.31,0.34,2.2,0.045,20.0,77.0,0.99270,3.30,0.43,10.2,5
348,7.8,0.18,0.46,13.6,0.052,38.0,118.0,0.99800,3.15,0.50,10.0,6
2272,6.2,0.25,0.28,8.5,0.035,28.0,108.0,0.99486,3.40,0.42,10.4,6
892,5.9,0.26,0.30,1.0,0.036,38.0,114.0,0.99280,3.58,0.48,9.4,5


## 2. MLflow server

Start the MLflow tracking server by 

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host localhost
    --port 5000
```

In [10]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host localhost --port 5000
remote_server_uri = "http://localhost:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [9]:
mlflow.tracking.get_tracking_uri()

'http://localhost:5000'

In [11]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/03/15 01:25:23 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [12]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [14]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
Save to: mlruns/1/360840b295de4a999aed17dd57601b2c/artifacts


In [15]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093895
  R2: 0.20224631822892103
Save to: mlruns/1/8e6b5b09acad49aaa5236f646b5ac24d/artifacts


In [16]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/1/a42a51c855614d0bb2de7f1b97d1e93c/artifacts
